In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200928


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [4]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

Inputファイル設定

In [56]:
#承認月
approval_month = "9月"

#①承認リスト（当月分）
input_file_1 = "N029（2020年9月1日～23日入会）.xlsx"
#②承認リスト（前月分）
input_file_2 = "N029（2020年8月入会）.xlsx"
#②承認リスト（前々月）追加
input_file_4 = "N029（2020年7月入会）.xlsx"
#②承認リスト（3カ月前）追加
input_file_5 = "N029（2020年6月入会）.xlsx"

#③ デジガレ確認用リスト
input_file_3 = "20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx"

Outputファイル定義

In [6]:
#Output_file
#①オンライン承認リスト
output_file_1 = f"当月オンライン承認リスト_{today_yyyymmdd}" 
print("Output①⇒",output_file_1)

#②アフィリエイト月次成果承認確定リスト
output_file_2 = f"デジガレ成果判定データ_{today_yyyymmdd}" 
print("Output②⇒",output_file_2)

Output①⇒ 当月オンライン承認リスト_20200928
Output②⇒ デジガレ成果判定データ_20200928


①承認リスト（当月分）を読み込む

In [7]:
print(input_file_1)

df_app = pd.read_excel(data_dir /r"input"/f"{input_file_1}")
df_app.head()

N029（2020年9月1日～23日入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200901,1,0,D00,8,31,1,0,8,107,2,0,NaN,NaN,NaN,NaN,155203017878797,55,12.0,7000,20.0,7.000401e+09,2,0
1,20200901,1,1,D02,1,2,1,0,3,103,2,0,NaN,NaN,NaN,NaN,155603602494324,53,12.0,3000,NaN,3.000010e+09,2,0
2,20200901,1,0,D00,1,31,1,0,8,102,2,0,NaN,NaN,NaN,NaN,155203017879712,48,12.0,2000,20.0,2.000410e+09,2,0
3,20200901,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921991085,38,52.0,10000,40.0,1.000000e+04,2,1
4,20200901,1,0,D00,4,31,2,0,3,299,1,0,NaN,NaN,NaN,NaN,155051922005075,51,12.0,11000,40.0,1.100000e+04,2,0


In [9]:
print(df_app.shape)
print(df_app.index)
print(df_app.columns)
print(df_app.dtypes)

(4446, 24)
RangeIndex(start=0, stop=4446, step=1)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア     float64
エンボス内容         float64
カード年会費CD         int64
リボ利用対象CD         int64
dtype: object


オンライン入会客を特定

In [10]:
df_app =df_app[(df_app["会員カード発行区分"] == "1") 
               & (df_app["続柄"] == 0) 
               & ~(df_app["カード提携会社2CD"] == 95) 
               & (df_app["切替CD"] == 0) 
               & (df_app["受付部支店"] == 299)]
#含まれてないか確認
print(df_app["会員カード発行区分"].unique())
print(df_app["続柄"].unique())
print(df_app["カード提携会社2CD"].unique())
print(df_app["切替CD"].unique())
print(df_app["受付部支店"].unique())
df_app.shape[0]

['1']
[0]
[ 1  4  8 11 14]
[0]
[299]


577

In [11]:
#"成果"カラムを追加
df_app['成果'] = "可"
df_app['参照用'] = df_app['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_app=df_app[['入会申込番号','成果','カード加入日','参照用']]
df_app.head()

,入会申込番号,成果,カード加入日,参照用
3,155051921991085,可,20200901,155051921991085
4,155051922005075,可,20200901,155051922005075
7,155051921984189,可,20200901,155051921984189
15,155051921991556,可,20200901,155051921991556
18,155051921990830,可,20200901,155051921990830


CSVアウトプット①

In [12]:
df_tmp = df_app.drop(columns=['参照用'])
df_tmp.to_csv(data_dir /r"output"/f"{output_file_1}.csv", index=False,header=True,encoding='shift-jis')

②前月の承認結果作業を読み込む

In [13]:
print(input_file_2)

df_pre = pd.read_excel(data_dir /r"input"/f"{input_file_2}")
df_pre.head()

N029（2020年8月入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200801,D,0,D00,50,31,0,0,8,103,2,0,NaN,NaN,NaN,NaN,155203047823789,55,12.0,3000,20.0,3.000327e+09,1,0
1,20200801,D,0,D00,50,31,0,0,8,107,2,0,NaN,NaN,NaN,NaN,155203037822197,48,12.0,7000,20.0,7.000401e+09,1,0
2,20200801,D,0,D00,50,31,0,0,8,106,2,0,NaN,NaN,NaN,NaN,155203037823401,28,12.0,6000,20.0,6.000411e+09,1,0
3,20200801,D,0,D00,50,31,0,0,8,107,2,0,NaN,NaN,NaN,NaN,155203037822080,40,12.0,7000,20.0,7.000000e+03,1,0
4,20200801,D,0,D00,50,31,0,0,8,107,2,0,NaN,NaN,NaN,NaN,155203047822625,40,12.0,7000,20.0,7.000404e+09,1,0


オンライン入会客を特定

In [14]:
df_pre =df_pre[(df_pre["会員カード発行区分"] == "1") 
               & (df_pre["続柄"] == 0) 
               & ~(df_pre["カード提携会社2CD"] == 95) 
               & (df_pre["切替CD"] == 0) 
               & (df_pre["受付部支店"] == 299)]
#含まれてないか確認
print(df_pre["会員カード発行区分"].unique())
print(df_pre["続柄"].unique())
print(df_pre["カード提携会社2CD"].unique())
print(df_pre["切替CD"].unique())
print(df_pre["受付部支店"].unique())
df_pre.shape[0]

['1']
[0]
[ 1  4  8 14 18 11]
[0]
[299]


681

In [15]:
#"成果"カラムを追加
df_pre['成果'] = "可"
df_pre['参照用'] = df_pre['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_pre=df_pre[['入会申込番号','成果','カード加入日','参照用']]
df_pre.head()

,入会申込番号,成果,カード加入日,参照用
42,155051921980070,可,20200803,155051921980070
58,155051921967986,可,20200803,155051921967986
62,155051921965022,可,20200803,155051921965022
70,155051921954794,可,20200803,155051921954794
80,155051921964405,可,20200803,155051921964405


前月分と当月分の承認リストを縦結合

In [16]:
df_app = pd.concat([df_pre, df_app])
df_app.shape[0]

1258

In [17]:
print(df_app['カード加入日'].unique())

[20200803 20200804 20200805 20200806 20200807 20200811 20200812 20200813
 20200814 20200817 20200818 20200819 20200820 20200821 20200824 20200825
 20200826 20200827 20200828 20200831 20200901 20200902 20200903 20200904
 20200907 20200908 20200909 20200910 20200911 20200914 20200915 20200916
 20200917 20200918 20200923]


②-2前々月の承認結果作業を読み込む　追加対応

In [18]:
print(input_file_4)

df_pre = pd.read_excel(data_dir /r"input"/f"{input_file_4}")
df_pre.head()

N029（2020年7月入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200701,1,0,D00,8,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921910960,46,12.0,7000,40.0,7.000000e+03,2,0
1,20200701,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921918211,50,52.0,6000,40.0,6.000000e+03,2,1
2,20200701,2,1,D00,8,31,1,0,3,1,1,0,NaN,NaN,NaN,NaN,155051921944480,58,52.0,2000,NaN,2.000310e+09,0,1
3,20200701,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602468245,36,12.0,2000,NaN,2.000010e+09,2,0
4,20200701,5,0,D00,95,0,0,0,3,1,2,0,NaN,NaN,NaN,NaN,155603602537692,47,NaN,0,NaN,NaN,1,0


オンライン入会客を特定

In [19]:
df_pre =df_pre[(df_pre["会員カード発行区分"] == "1") 
               & (df_pre["続柄"] == 0) 
               & ~(df_pre["カード提携会社2CD"] == 95) 
               & (df_pre["切替CD"] == 0) 
               & (df_pre["受付部支店"] == 299)]
#含まれてないか確認
print(df_pre["会員カード発行区分"].unique())
print(df_pre["続柄"].unique())
print(df_pre["カード提携会社2CD"].unique())
print(df_pre["切替CD"].unique())
print(df_pre["受付部支店"].unique())
df_pre.shape[0]

['1']
[0]
[ 8  1  4 18 14 11]
[0]
[299]


889

In [20]:
#"成果"カラムを追加
df_pre['成果'] = "可"
df_pre['参照用'] = df_pre['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_pre=df_pre[['入会申込番号','成果','カード加入日','参照用']]
df_pre.head()

,入会申込番号,成果,カード加入日,参照用
0,155051921910960,可,20200701,155051921910960
1,155051921918211,可,20200701,155051921918211
5,155051921922544,可,20200701,155051921922544
11,155051921940074,可,20200701,155051921940074
31,155051921944332,可,20200701,155051921944332


前々月分の承認リストを縦結合

In [21]:
df_app = pd.concat([df_pre, df_app])
df_app.shape[0]

2147

In [22]:
print(df_app['カード加入日'].unique())

[20200701 20200702 20200703 20200706 20200707 20200708 20200709 20200710
 20200713 20200714 20200715 20200716 20200717 20200720 20200721 20200722
 20200727 20200728 20200729 20200730 20200731 20200803 20200804 20200805
 20200806 20200807 20200811 20200812 20200813 20200814 20200817 20200818
 20200819 20200820 20200821 20200824 20200825 20200826 20200827 20200828
 20200831 20200901 20200902 20200903 20200904 20200907 20200908 20200909
 20200910 20200911 20200914 20200915 20200916 20200917 20200918 20200923]


②-3 3カ月の承認結果作業を読み込む　追加対応（過渡期対応）

In [57]:
print(input_file_5)

df_pre = pd.read_excel(data_dir /r"input"/f"{input_file_5}")
df_pre.head()

N029（2020年6月入会）.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD
0,20200601,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921903874,37,12.0,2000,40.0,2.000000e+03,2,0
1,20200601,1,0,D02,4,2,2,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336764,52,12.0,2000,NaN,2.000010e+09,2,0
2,20200601,1,0,D02,4,2,2,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602336533,43,12.0,2000,NaN,2.000010e+09,2,0
3,20200601,1,1,D02,1,2,1,0,3,102,2,0,NaN,NaN,NaN,NaN,155603602336251,48,12.0,2000,NaN,2.000010e+09,2,0
4,20200601,D,0,D00,50,31,0,0,8,103,2,0,NaN,NaN,NaN,NaN,155203047717981,71,12.0,3000,20.0,3.000079e+09,1,0


オンライン入会客を特定

In [58]:
df_pre =df_pre[(df_pre["会員カード発行区分"] == "1") 
               & (df_pre["続柄"] == 0) 
               & ~(df_pre["カード提携会社2CD"] == 95) 
               & (df_pre["切替CD"] == 0) 
               & (df_pre["受付部支店"] == 299)]
#含まれてないか確認
print(df_pre["会員カード発行区分"].unique())
print(df_pre["続柄"].unique())
print(df_pre["カード提携会社2CD"].unique())
print(df_pre["切替CD"].unique())
print(df_pre["受付部支店"].unique())
df_pre.shape[0]

['1']
[0]
[ 1  8  4 11 14]
[0]
[299]


697

In [59]:
#"成果"カラムを追加
df_pre['成果'] = "可"
df_pre['参照用'] = df_pre['入会申込番号'] #"参照用"はデジガレとのマッチング用にリネーム

df_pre=df_pre[['入会申込番号','成果','カード加入日','参照用']]
df_pre.head()

,入会申込番号,成果,カード加入日,参照用
0,155051921903874,可,20200601,155051921903874
19,155051921903460,可,20200601,155051921903460
35,155051921902660,可,20200601,155051921902660
38,155051921902561,可,20200601,155051921902561
47,155051921878019,可,20200601,155051921878019


前々月分の承認リストを縦結合

In [60]:
df_app = pd.concat([df_pre, df_app])
df_app.shape[0]

2844

In [61]:
print(df_app['カード加入日'].unique())

[20200601 20200602 20200603 20200604 20200605 20200608 20200609 20200610
 20200611 20200612 20200615 20200616 20200617 20200618 20200619 20200622
 20200623 20200624 20200625 20200626 20200629 20200630 20200701 20200702
 20200703 20200706 20200707 20200708 20200709 20200710 20200713 20200714
 20200715 20200716 20200717 20200720 20200721 20200722 20200727 20200728
 20200729 20200730 20200731 20200803 20200804 20200805 20200806 20200807
 20200811 20200812 20200813 20200814 20200817 20200818 20200819 20200820
 20200821 20200824 20200825 20200826 20200827 20200828 20200831 20200901
 20200902 20200903 20200904 20200907 20200908 20200909 20200910 20200911
 20200914 20200915 20200916 20200917 20200918 20200923]


③ デジガレからの成果確認依頼リスト

In [62]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 0)

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file_3}")
print(bk.sheet_names)

df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）', '成果データ（5月～利用条件追加媒体）_承認返却済み']


,ASID,メディア名,媒体チャネル,クリック,申込,申込月,参照用,ASP名,単価,成果,カード発行日
0,49.0,ハピタス（連携用）,ポイント,2020-05-28 12:54:13,2020-05-28 13:11:05,43952,155051921900045,BGT,7000,保留,保留
1,49.0,ハピタス（連携用）,ポイント,2020-05-28 13:52:57,2020-05-28 14:02:05,43952,155051921900110,BGT,7000,保留,保留
2,619.0,モッピー（連携用）,ポイント,2020-05-28 14:24:02,2020-05-28 14:38:04,43952,155051921900169,BGT,7000,保留,保留
3,49.0,ハピタス（連携用）,ポイント,2020-05-28 14:36:08,2020-05-28 14:45:04,43952,155051921900185,BGT,7000,保留,保留
4,619.0,モッピー（連携用）,ポイント,2020-05-28 14:41:20,2020-05-28 14:51:06,43952,155051921900201,BGT,7000,保留,保留


In [63]:
#シート名の確認
print(bk.sheet_names[0])

成果データ


「成果データ」シート分の対応

In [64]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])

In [66]:
#ASIDがfloatになっていまうので整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(1366, 9)
RangeIndex(start=0, stop=1366, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '申込月', '参照用', 'ASP名', '単価'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
申込月                int64
参照用                int64
ASP名              object
単価                 int64
dtype: object


In [67]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,49,ハピタス（連携用）,ポイント,2020-05-28 12:54:13,2020-05-28 13:11:05,155051921900045,BGT,7000,2020年5月
1,49,ハピタス（連携用）,ポイント,2020-05-28 13:52:57,2020-05-28 14:02:05,155051921900110,BGT,7000,2020年5月
2,619,モッピー（連携用）,ポイント,2020-05-28 14:24:02,2020-05-28 14:38:04,155051921900169,BGT,7000,2020年5月
3,49,ハピタス（連携用）,ポイント,2020-05-28 14:36:08,2020-05-28 14:45:04,155051921900185,BGT,7000,2020年5月
4,619,モッピー（連携用）,ポイント,2020-05-28 14:41:20,2020-05-28 14:51:06,155051921900201,BGT,7000,2020年5月


デジガレデータに承認データの"成果"をマージ

In [68]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
16,49,ハピタス（連携用）,ポイント,2020-05-29 20:32:19,2020-05-29 20:50:06,155051921901720,BGT,7000,2020年5月,可,20200825.0
156,619,モッピー（連携用）,ポイント,2020-07-01 22:38:21,2020-07-01 22:56:07,155051921945545,BGT,7000,2020年7月,可,20200825.0
169,619,モッピー（連携用）,ポイント,2020-07-06 13:59:38,2020-07-06 14:10:11,155051921953481,BGT,7000,2020年7月,可,20200903.0
172,5953,ポイントタウン（JIPC参加企業）（連携用）,ポイント,2020-07-06 19:34:49,2020-07-06 19:47:11,155051921954018,BGT,7000,2020年7月,可,20200918.0
180,49,ハピタス（連携用）,ポイント,2020-07-07 18:35:47,2020-07-07 18:50:10,155051921955726,BGT,7000,2020年7月,可,20200827.0


In [70]:
df_dg2 = df_dg2.copy()

#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

In [71]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,49,ハピタス（連携用）,ポイント,2020-05-28 12:54:13,2020-05-28 13:11:05,155051921900045,BGT,7000,2020年5月,NaN,0
1,49,ハピタス（連携用）,ポイント,2020-05-28 13:52:57,2020-05-28 14:02:05,155051921900110,BGT,7000,2020年5月,NaN,0
2,619,モッピー（連携用）,ポイント,2020-05-28 14:24:02,2020-05-28 14:38:04,155051921900169,BGT,7000,2020年5月,NaN,0
3,49,ハピタス（連携用）,ポイント,2020-05-28 14:36:08,2020-05-28 14:45:04,155051921900185,BGT,7000,2020年5月,NaN,0
4,619,モッピー（連携用）,ポイント,2020-05-28 14:41:20,2020-05-28 14:51:06,155051921900201,BGT,7000,2020年5月,NaN,0


In [73]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "否認"#保留から変更 20/8/25
        

#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "否認"#保留から変更 20/8/25

In [74]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日
0,49,ハピタス（連携用）,ポイント,2020-05-28 12:54:13,2020-05-28 13:11:05,155051921900045,BGT,7000,2020年5月,否認,否認
1,49,ハピタス（連携用）,ポイント,2020-05-28 13:52:57,2020-05-28 14:02:05,155051921900110,BGT,7000,2020年5月,否認,否認
2,619,モッピー（連携用）,ポイント,2020-05-28 14:24:02,2020-05-28 14:38:04,155051921900169,BGT,7000,2020年5月,否認,否認
3,49,ハピタス（連携用）,ポイント,2020-05-28 14:36:08,2020-05-28 14:45:04,155051921900185,BGT,7000,2020年5月,否認,否認
4,619,モッピー（連携用）,ポイント,2020-05-28 14:41:20,2020-05-28 14:51:06,155051921900201,BGT,7000,2020年5月,否認,否認


集計

In [75]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

405

In [76]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,可,否認
申込年月,,
2020年1月,0,3
2020年2月,0,2
2020年5月,1,33
2020年6月,1,122
2020年7月,31,103
2020年8月,58,75
2020年9月,314,623


CSVアウトプット②

In [38]:
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[0]}.csv", index=False,header=True,encoding='shift-jis')

利用条件による成果判定チェックは別ノートブックで
<br>ここでは単純に加入有無だけを確認

In [77]:
print(bk.sheet_names[2])

成果データ（5月～利用条件追加媒体）


③ デジガレ「利用条件」リスト

In [78]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 2)
df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日,カード売上額合計
0,21.0,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,保留,保留,1.550519e+14
1,128.0,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,保留,保留,1.550519e+14
2,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,保留,保留,1.550519e+14
3,57.0,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,保留,20200624,1.550519e+14
4,21.0,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,保留,保留,1.550519e+14


In [79]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])
df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計
0,21.0,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14
1,128.0,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14
2,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14
3,57.0,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14
4,21.0,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14


In [81]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(918, 9)
RangeIndex(start=0, stop=918, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価',
       'カード売上額合計'],
      dtype='object')
ASID                 int32
メディア名               object
媒体チャネル              object
クリック        datetime64[ns]
申込          datetime64[ns]
参照用                  int64
ASP名                object
単価                   int64
カード売上額合計           float64
dtype: object


In [82]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月
0,21,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14,2020年6月
1,128,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14,2020年6月
2,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14,2020年6月
3,57,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14,2020年6月
4,21,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14,2020年6月


In [83]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月,成果,カード加入日
3,57,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14,2020年6月,可,20200624.0
7,57,ライフメディア,ポイント,2020-06-02 17:18:21,2020-06-02 17:34:12,155051921906406,BGT,9000,1.550519e+14,2020年6月,可,20200623.0
9,37,げん玉,ポイント,2020-06-03 23:45:25,2020-06-04 00:03:06,155051921907750,BGT,9000,1.550519e+14,2020年6月,可,20200622.0
13,128,Warau.jp,ポイント,2020-06-04 11:13:45,2020-06-04 11:24:06,155051921907941,BGT,9000,1.550519e+14,2020年6月,可,20200624.0
18,128,Warau.jp,ポイント,2020-06-04 18:09:23,2020-06-04 18:26:07,155051921908345,BGT,9000,1.550519e+14,2020年6月,可,20200623.0


In [84]:
df_dg2 = df_dg2.copy()

In [86]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月,成果,カード加入日
0,21,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14,2020年6月,NaN,0
1,128,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14,2020年6月,NaN,0
2,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14,2020年6月,NaN,0
4,21,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14,2020年6月,NaN,0
5,21,ポイントインカム,ポイント,2020-06-01 22:58:32,2020-06-01 23:18:11,155051921905911,BGT,9000,1.550519e+14,2020年6月,NaN,0


In [88]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"
        
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "保留"

In [89]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月,成果,カード加入日
0,21,ポイントインカム,ポイント,2020-06-01 01:25:55,2020-06-01 01:34:29,155051921904849,BGT,9000,1.550519e+14,2020年6月,保留,保留
1,128,Warau.jp,ポイント,2020-06-01 09:42:13,2020-06-01 09:50:12,155051921904930,BGT,9000,1.550519e+14,2020年6月,保留,保留
2,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-06-01 12:35:04,2020-06-01 12:58:12,155051921905143,BGT,9000,1.550519e+14,2020年6月,保留,保留
3,57,ライフメディア,ポイント,2020-06-01 16:52:21,2020-06-01 17:03:12,155051921905432,BGT,9000,1.550519e+14,2020年6月,可,20200624
4,21,ポイントインカム,ポイント,2020-06-01 22:16:51,2020-06-01 22:26:12,155051921905846,BGT,9000,1.550519e+14,2020年6月,保留,保留


集計

In [90]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

727

In [91]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年6月,78,105
2020年7月,68,459
2020年8月,45,163


CSVアウトプット

In [93]:
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[2]}.csv", index=False,header=True,encoding='shift-jis')

利用条件による成果判定チェックは別ノートブックで（その２）
<br>ここでは単純に加入有無だけを確認

In [94]:
print(bk.sheet_names[3])

成果データ（5月～利用条件追加媒体）_承認返却済み


③ デジガレ「利用条件」リスト

In [95]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 3)
df_dg.head()

20200925〆【JFRカード御中】成果データ&問い合わせ.xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,保留,20200615,NaN
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,保留,20200611,NaN
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,保留,20200525,NaN
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,保留,20200525,NaN
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,保留,20200526,NaN


In [100]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])
df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,NaN,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,NaN,2020年5月
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,NaN,2020年5月


In [101]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(250, 10)
RangeIndex(start=0, stop=250, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価',
       'カード売上額合計', '申込年月'],
      dtype='object')
ASID                 int32
メディア名               object
媒体チャネル              object
クリック        datetime64[ns]
申込          datetime64[ns]
参照用                  int64
ASP名                object
単価                   int64
カード売上額合計           float64
申込年月                object
dtype: object


In [102]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,NaN,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,NaN,2020年5月
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,NaN,2020年5月


In [103]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月,可,20200615.0
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月,可,20200611.0
6,37,げん玉,ポイント,2020-05-01 10:59:02,2020-05-01 11:14:05,155051921869646,BGT,9000,NaN,2020年5月,可,20200616.0
9,37,げん玉,ポイント,2020-05-01 13:52:48,2020-05-01 14:05:07,155051921869869,BGT,9000,NaN,2020年5月,可,20200615.0
12,37,げん玉,ポイント,2020-05-01 17:19:07,2020-05-01 17:39:04,155051921870339,BGT,9000,NaN,2020年5月,可,20200602.0


In [104]:
df_dg2 = df_dg2.copy()

In [106]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月,成果,カード加入日
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,NaN,2020年5月,NaN,0
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,NaN,2020年5月,NaN,0
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,NaN,2020年5月,NaN,0
5,21,ポイントインカム,ポイント,2020-05-01 10:27:46,2020-05-01 10:37:05,155051921869588,BGT,9000,NaN,2020年5月,NaN,0
7,37,げん玉,ポイント,2020-05-01 12:20:52,2020-05-01 12:34:05,155051921869760,BGT,9000,NaN,2020年5月,NaN,0


In [108]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "否認"#保留から変更 20/8/25
        
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "否認"#保留から変更 20/8/25

In [109]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,カード売上額合計,申込年月,成果,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,NaN,2020年5月,可,20200615
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,NaN,2020年5月,可,20200611
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,NaN,2020年5月,否認,否認
3,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,NaN,2020年5月,否認,否認
4,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,NaN,2020年5月,否認,否認


集計

In [110]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

116

In [111]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,可,否認
申込年月,,
2020年5月,62,134
2020年6月,54,0


CSVアウトプット

In [112]:
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[3]}.csv", index=False,header=True,encoding='shift-jis')